# Interactive KBO Notebook
In this notebook, we will make a bare-bones application to interactively explore the movement and light profile of KBOs and other moving objects. To fetch data, we will use the [Solar System Object Image Search](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/ssois/documentation.html) tool provided by the CADC.


## Table of Contents
* [1. Introduction](#1.-Introduction)
* [2. Setup](#2.-Setup)
* [3. SSOIS Query](#3.-SSOIS-Query)
    * [3.1 Aladin Widget](#3.1-Aladin-Widget)
* [4. Helper Functions](#4.-Helper-Functions)
* [5. Widgets](#5.-Widgets)
    * [5.1 The KBO Figure](#5.1-The-KBO-Figure)
* [6. Putting it All Together](#6.-Putting-it-All-Together)

## 1. Introduction

The idea for this notebook is to give an outline of a lightweight, interactive tool in a jupyter notebook. We want the ability to display, interact with, and analyze data interactively. The end goal is an application that allows a user to select a subset of SSOIS images using a date range slider and plot the subset, then select a circular area on the interactive plot and calculate the aperture sum.

## 2. Setup
This tutorial will go through some of the basic functionalities of the python packages such as:
* _astropy_ for astronomical operations
* _ipyaladin_ for a widget for Aladin Lite
* _ipywidgets_ to build interactive widgets
* _matplotlib_ to plot data
* _ipympl_ for the interactive `%matplotlib widget` backend

## 3. SSOIS Query
In order to get images of a moving object, we will use the [Solar System Object Image Search](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/ssois/documentation.html) tool provided by the CADC. We will search by object name between a start and end date. To choose an object to search, we will use the [Minor Planet Center search tool](https://minorplanetcenter.net/db_search/) and search the database where the minimum semi-major axis value is 50 AU. From the [results table](https://minorplanetcenter.net/db_search/show_by_properties?utf8=%E2%9C%93&semimajor_axis_min=50&semimajor_axis_max=&eccentricity_min=&eccentricity_max=&inclination_min=&inclination_max=&argument_of_perihelion_min=&argument_of_perihelion_max=&ascending_node_min=&ascending_node_max=&mean_anomaly_min=&mean_anomaly_max=&mean_daily_motion_min=&mean_daily_motion_max=&perihelion_distance_min=&perihelion_distance_max=&aphelion_distance_min=&aphelion_distance_max=&period_min=&period_max=&absolute_magnitude_min=&absolute_magnitude_max=&orbit_uncertainty_min=&orbit_uncertainty_max=), we chose [the KBO 2010 GB174](https://minorplanetcenter.net/db_search/show_object?object_id=2010+GB174) to query for. We will search between the dates January 1st, 2010, to January 1st, 2016.

In [1]:
%matplotlib widget
from urllib.parse import urlencode

import pandas as pd

# Define search parameters
kbo_name = '2010 GB174'
start_date = '2010 01 01'
end_date = '2016 01 01'

# Build query url
params = {
    'lang': 'en',
    'object': kbo_name,
    'search': 'bynameCADC',
    'epoch1': start_date,
    'epoch2': end_date,
    'eunits': 'arcseconds',
    'extres': 'no',  # Return the extension where the KBO can be seen in
    'xyres': 'no',  # Return the pixel X and Y position of the KBO
    'format': 'tsv'
}
base_url = 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/cadcbin/ssos/ssosclf.pl'
url = '{}?{}'.format(base_url, urlencode(params))

# Access data and convert to panadas dataframe
data_table = pd.read_csv(url, sep='\t')

# Get results with a long exposure and the g-band filter
data_table = data_table[(data_table['Exptime'] > 100)
                        & (data_table['Filter'] == "G.MP9401")]

data_table

,Image,MJD,Filter,Exptime,Object_RA,Object_Dec,Image_target,Telescope/Instrument,MetaData,Datalink
17,1183654p,55298.348651,G.MP9401,634,189.622392,15.045956,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
18,1183824p,55299.294388,G.MP9401,634,189.611366,15.049740,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
19,1183831p,55299.351782,G.MP9401,634,189.610699,15.049966,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
20,1183838p,55299.409538,G.MP9401,634,189.610028,15.050193,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
21,1183845p,55299.466308,G.MP9401,634,189.609369,15.050416,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN


### 3.1 Aladin Widget
To show the location of the SSOIS results, we can use the interactive widget provided by Aladin. This widget has much more functionality that can be explored in the [jupyter notebooks on the ipyaladin github](https://github.com/cds-astro/ipyaladin/tree/master/examples).

In [2]:
import numpy as np
from astropy.table import Column, Table
from ipywidgets import Layout

from ipyaladin import Aladin

# Convert data table to astropy table and rename RA and Dec columns
table = Table.from_pandas(data_table)
table.rename_column('Object_RA', 'RA')
table.rename_column('Object_Dec', 'DEC')
table.rename_column('Telescope/Instrument', 'Telescope_Instrument')
table['Image'] = Column(data=data_table['Image'],
                        name='Image',
                        dtype=np.dtype('S10'))

target_pos = '{} {}'.format(table[0]['RA'], table[0]['DEC'])

# Display the widget and add the table
aladin = Aladin(target=target_pos,
                layout=Layout(width='100%', height='400px'),
                fov=1)
display(aladin)
aladin.add_table(table)

Aladin(fov=1.0, layout=Layout(height='400px', width='100%'), options=['allow_full_zoomout', 'coo_frame', 'fov'…

### 3.2 Get CADC data
Since we want to later use the datalink service to fetch cutouts of the KBOs, we will query CADC for the results to get the publisherIDs.


In [3]:
from astroquery.cadc import Cadc

cadc = Cadc()

cadc_results = cadc.exec_sync('''SELECT upload_table.*, Plane.publisherID 
FROM tap_upload.upload_table AS upload_table
JOIN caom2.Plane AS Plane ON Plane.productID = upload_table.Image
JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID ''',
                              uploads={'upload_table': table[['Image', 'MJD', 'RA', 'DEC']]})

cadc_table = cadc_results.to_pandas()
str_results = cadc_table[['Image', 'publisherID']
                         ].stack().str.decode('utf-8').unstack()
for col in str_results:
    cadc_table[col] = str_results[col]

cadc_table

,Image,MJD,RA,DEC,publisherID
0,1183654p,55298.348651,189.622392,15.045956,ivo://cadc.nrc.ca/CFHT?1183654/1183654p
1,1183831p,55299.351782,189.610699,15.049966,ivo://cadc.nrc.ca/CFHT?1183831/1183831p
2,1183845p,55299.466308,189.609369,15.050416,ivo://cadc.nrc.ca/CFHT?1183845/1183845p
3,1183824p,55299.294388,189.611366,15.049740,ivo://cadc.nrc.ca/CFHT?1183824/1183824p
4,1183838p,55299.409538,189.610028,15.050193,ivo://cadc.nrc.ca/CFHT?1183838/1183838p


## 4. Helper Functions
The end goal is to make an application with a date slider that selects a subset of data then plots all the files in the subset, with interactivity to preform aperture photometry. To speed up the program, we can use cutouts to only show the data that we want to look at. In order to do all of this, we need helper functions to select a subset of images, calculate the cutout parameters, get the cutout urls, and fetch the cutout data.

In [4]:
import re
from urllib.error import HTTPError

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.time import Time
from astropy.wcs import WCS


def get_selection(table, date_range):
    """Returns the subset of rows in the table within the date_range."""
    return table[(table['MJD'] >= date_range[0])
                 & (table['MJD'] < date_range[1] + 1)]


def get_cutout_params(table, min_radius=0.005):
    """Given a table with R.A. and Dec. columns, returns the parameters of the
    circle needed to include all positions."""

    # Get the max and min R.A. and Dec. values
    ra_max, ra_min = table['RA'].max(), table['RA'].min()
    dec_max, dec_min = table['DEC'].max(), table['DEC'].min()

    # Get the R.A. and Dec. centre and radius that covers all points
    ra_centre = (ra_max + ra_min) / 2
    dec_centre = (dec_max + dec_min) / 2
    ra_radius = abs(ra_max - ra_min) / 2
    dec_radius = abs(dec_max - dec_min) / 2
    
    coords = SkyCoord(ra_centre, dec_centre, unit=(u.deg, u.deg))
    radius = max(round(max(ra_radius, dec_radius), 2), min_radius) * u.deg

    return coords, radius


def get_data_list(table, ext=0, verbose=False):
    """Given the table with cutout urls, return the fits image and WCS information"""
    data_list = []
    if len(table) < 1:
        print("Range is too small")
        return None
    coords, radius = get_cutout_params(table)

    table['cutout_url'] = cadc.get_image_list(table, coords, radius)

    for url, ra, dec, date in zip(table['cutout_url'], table['RA'],
                                  table['DEC'], table['MJD']):
        try:
            with fits.open(url, ignore_missing_end=True) as hdulist:
                if verbose:
                    hdulist.info()
                data = hdulist[ext].data
                wcs = WCS(hdulist[ext].header)
                date_fmtd = Time(date, format='mjd', out_subfmt='date_hm').iso
                data_list.append({
                    'image_data': data,
                    'wcs': wcs,
                    'ra': ra,
                    'dec': dec,
                    'date': date_fmtd
                })
        except HTTPError as ex:
            print(ex)
            continue
    return data_list

## 5. Widgets
Now we make our widgets! We want one widget to allow the user to select a date range to display, a button to replot the data, and another widget to display the aperture data. Technically, we also want the plot itself to be a widget, but we will come to that later.

In [5]:
import math

import numpy as np
from ipywidgets import widgets
from traitlets import traitlets


# Date range slider widget
def make_date_range_slider(table):
    """ Function to generate the date range slider based on the date ranges in the results table"""
    start_date = math.floor(table['MJD'].min())
    end_date = math.ceil(table['MJD'].max())

    dates = Time(np.arange(start_date, end_date, 1),
                 format='mjd',
                 out_subfmt='date')
    # Match value to formated date output for nicer display
    options = [(' {} '.format(date.iso), date.mjd) for date in dates]

    # Range slider to select dates
    range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=(0, 1),
        description='Date Range:',
        disabled=False,
        continuous_update=True,
        layout=Layout(width='700px'),
    )

    # The slider controls the selected rows of the table data
    selection = get_selection(table, range_slider.value)
    range_slider.add_traits(selection=traitlets.Any(selection))

    # Label to show number of files in selected date range
    range_label = widgets.Label(
        value="Number of Results: {}".format(len(selection)))

    # When slider changes, update selection table and results num label
    def update_selection(change):
        new_selection = get_selection(table, change['new'])
        range_slider.selection = new_selection
        range_label.value = "Number of Results: {}".format(len(new_selection))

    range_slider.observe(update_selection, names='value')

    return widgets.VBox([range_slider, range_label])


# Replot button widget
def make_replot_button(range_slider, kbo_fig, kbo_name):
    replot_button = widgets.Button(description="Replot")
    
    # When button is clicked, get new selection and replot
    def on_button_clicked(change):
        data_list = get_data_list(Table.from_pandas(range_slider.children[0].selection), ext='ccd17')
        plot_data(kbo_fig, data_list, kbo_name)

    replot_button.on_click(on_button_clicked)
    
    return replot_button


# Aperture results widget
def make_aperture_text_widget():
    """Returns widget to display aperture information"""
    aperture_widget = widgets.HTML(
        value='Click and drag on a subplot to select an area to analyze',
        placeholder='<b>Aperture Sum: </b>',
        description='<b>Aperture Sum: </b>',
        disabled=False,
        style={'description_width': 'initial'},
        layout=Layout(width='90%'),
    )
    return aperture_widget

### 5.1 The KBO Figure
This is the interactive plot of the KBO. Since the `matplotlib widget` backend is used, the figure canvas can be used as a widget, which we will use in a box along with the aperture widget. We want the ability to draw circles to show which area is going to be used in the aperture analysis, meaning we will have to use click event handlers.

In [6]:
import warnings

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
from matplotlib.patches import Circle
from photutils import CircularAperture, SkyAperture, aperture_photometry

# Supress fits processing warnings
warnings.simplefilter('ignore')


def kbo_figure(data_list, kbo_name):
    """Build the figure and click events"""
    global data_dict
    p_center = None
    data_dict = {}

    def onclick(event):
        global p_center
        
        # Return if in pan/zoom mode or click outside of axes
        if (plt.get_current_fig_manager().toolbar.mode != ''
                or event.inaxes is None):
            return
        
        p_center = (event.xdata, event.ydata)

    def onrelease(event):
        global p_center
        global kbo_fig
        global data_dict

        # Return if in pan/zoom mode or click outside of axes
        if (plt.get_current_fig_manager().toolbar.mode != ''
                or event.inaxes is None):
            return

        # Calculate circle radius
        p_outside = (event.xdata, event.ydata)
        radius = math.sqrt((p_center[0] - p_outside[0])**2 +
                           (p_center[1] - p_outside[1])**2)

        if radius == 0.0: return

        # Remove old circles
        for ax in kbo_fig.axes:
            for artist in ax.artists:
                if isinstance(artist, plt.Circle):
                    artist.remove()

        # Draw new circles
        circle = plt.Circle(p_center, radius, color='black', fill=False)
        ax = event.inaxes
        ax.add_artist(circle)
        kbo_fig.canvas.draw()

        # Do aperture photometry
        aperture = CircularAperture(p_center, r=radius)
        image_data = data_dict[(ax.rowNum, ax.colNum)]
        phot_table = aperture_photometry(image_data, aperture)
        aperture_widget.value = '{:.8}'.format(phot_table['aperture_sum'][0])

    # Create figure and oonnect click events
    kbo_fig = plt.figure(figsize=(9, 8))
    cid_press = kbo_fig.canvas.mpl_connect('button_press_event', onclick)
    cid_release = kbo_fig.canvas.mpl_connect('button_release_event', onrelease)

    plot_data(kbo_fig, data_list, kbo_name)
    return kbo_fig


def plot_data(kbo_fig, data_list, kbo_name, max_cols=3):
    """Plot data on datalist to KBO fig"""
    global data_dict
    data_dict = dict()
    n = len(data_list)
    
    # Clear the figure if needed
    plt.clf()

    kbo_fig.suptitle('Images of KBO {}'.format(kbo_name),
         fontsize=14,
         fontweight='bold')
        
    # Set all axes to be in same WCS
    wcs_trans = data_list[0]['wcs']
    gs = gridspec.GridSpec(math.ceil(n / max_cols),
                           max_cols,
                           figure=kbo_fig,
                           wspace=0.1)

    for idx, data in enumerate(data_list):
        try:

            
            ax = plt.subplot(gs[idx // max_cols, idx % max_cols],
                             projection=wcs_trans,
                             adjustable='box',
                             aspect='equal')
            
            # Set the global data_dict to be used for photometry
            data_dict[(ax.rowNum, ax.colNum)] = data['image_data']

            # Normalize and plot the data
            image_data_norm = ImageNormalize(data['image_data'],
                                             interval=ZScaleInterval(),
                                             stretch=LinearStretch())
            ax.imshow(data['image_data'],
                      norm=image_data_norm,
                      transform=ax.get_transform(data['wcs']),
                      cmap='gray')
            ax.set_title(data['date'], fontsize=10)

            # Add yellow circle around KBO position
            c = Circle((data['ra'], data['dec']),
                       0.002,
                       edgecolor='yellow',
                       facecolor='none',
                       transform=ax.get_transform('world'))
            ax.add_patch(c)

            # Add grid and remove axis labels
            ax.coords.grid(color='white', ls='solid')
            ra, dec = ax.coords['ra'], ax.coords['dec']
            ra.set_ticklabel_visible(False)
            dec.set_ticklabel_visible(False)
            ra.set_axislabel('')
            dec.set_axislabel('')

        except ValueError as ex:
            print('Value Error: %s' % ex)
            continue

    axes = kbo_fig.get_axes()
    axes[0].get_shared_x_axes().join(*axes)
    axes[0].get_shared_y_axes().join(*axes)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.9)
    plt.tight_layout(pad=0.5)

## 6. Putting it All Together
Now we can build the application. We can put the widgets together in a nice fashion using the HBox and VBox widgets then display everything.

In [7]:
from astropy.table import Table
range_slider = make_date_range_slider(cadc_table)
data_list = get_data_list(Table.from_pandas(range_slider.children[0].selection), ext='ccd17')

plt.ioff()
kbo_fig = kbo_figure(data_list, kbo_name)
plt.ion()

replot_button = make_replot_button(range_slider, kbo_fig, kbo_name)


In [8]:
# Organize the widgets
b_layout = widgets.Layout(align_items='stretch',
                          align_content='center',
                          border='none',
                          justify_content='center',
                          width='100%')

aperture_widget = make_aperture_text_widget()
silder_and_button = widgets.HBox([range_slider, replot_button], layout=b_layout)
big_box = widgets.VBox([silder_and_button, kbo_fig.canvas, aperture_widget], layout=b_layout)

display(big_box)

Use the interactive toolbar at the bottom of the plot to pan and zoom around. Note that each view is linked, so moving one axes view will move all the others. To select an area to preform an aperture sum, be sure to deselect the zoom/pan button then click in the centre of the area you want to analyze and drag out to the edge. The aperture sum label at the bottom should automatically be filled with the right value. Hopefully this tutorial has given you an understanding of how to build a simple astronomy application in a notebook. 